# Incremental R2 Implementation

In [ ]:
# !pip install sortedcontainers numpy pandas

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement moocore (from versions: none)
ERROR: No matching distribution found for moocore


In [1]:
import numpy as np
import pandas as pd
from sortedcontainers import SortedList

from r2_incremental import *

In [2]:
# test sorted list
sl = SortedList([(0.2,0.8), (0.1,0.9), (0.3,0.7)])
sl

SortedList([(0.1, 0.9), (0.2, 0.8), (0.3, 0.7)])

In [3]:
p = (0.1,0.9)
idx = sl.bisect_left(p)
print(f"{p}, {sl[idx]}")
dominates(p, sl[idx])

(0.1, 0.9), (0.1, 0.9)


False

In [8]:
# archive = [(x + 0.1 * y, (1-x) + 0.1  * y) for (x, y) in zip(np.random.random(10**6), np.random.random(10**6))]
num = 1 + 10**6
source = np.linspace(0, 1, num)
noise = 1 * np.random.random(num)
np.random.shuffle(source)
archive = [(x + y, (1 - x) + y) for x,y in zip(source, noise)]

archive = [(1,1), (0,1), (1,0), (0.5,0.5), (0.5,0.5), (0.5,0.51), (0.5,0.49), (0,2)]

In [9]:
pd.DataFrame(archive).sum(axis = 1).min()

0.99

In [25]:
utility(0, 1, 2)

0.0

In [11]:
r2_history, hv_history = compute_indicators_archive(archive, ideal = (0,0), ref = (1,1))

add (1, 1)
0.75
remove (1, 1)
0.75
add (0, 1)
0.5
add (1, 0)
-0.25
add (0.5, 0.5)
-0.041666666666666685
remove (0.5, 0.5)
-0.041666666666666685
add (0.5, 0.49)
-0.042641176869364805


In [ ]:
r2 = r2_history[-1]
hv = hv_history[-1]

In [8]:
r2_history

[0.75,
 0.5,
 0.25,
 0.20833333333333331,
 0.20833333333333331,
 0.20833333333333331,
 0.2073588231306352,
 0.2073588231306352]

In [10]:
r2

0.2073588231306352

In [11]:
(r2, r2 - 1/6)

(0.2073588231306352, 0.04069215646396854)

In [12]:
(hv, 1/2 - hv)

(0.4982195851077759, 0.0017804148922240803)

In [665]:
ind_df = pd.DataFrame(
    np.stack([list(range(1,len(r2_history) + 1)), r2_history, hv_history], axis = 1),
    columns = ["feval", "r2", "hv"]
)

In [666]:
ind_df.to_csv("ind_df.csv", index=False)

In [667]:
r2_history[0:6]

[0.7759953792260945,
 0.6371907055260099,
 0.49490549365409386,
 0.49490549365409386,
 0.49490549365409386,
 0.49490549365409386]

In [503]:
hv

0.5867338488109856

In [341]:
pd.DataFrame(np.stack(sl))

,0,1
0,0.000164,0.979773
1,0.000580,0.964566
2,0.000929,0.954876
3,0.001083,0.950795
4,0.001369,0.945396
...,...,...
683,0.954649,0.000531
684,0.964080,0.000479
685,0.972932,0.000214
686,0.977364,0.000152


In [342]:
(pd.DataFrame(np.stack(sl))[1].diff()[1:] < 0).mean()

1.0

In [343]:
np.stack(archive)

array([[0.16303393, 0.36069037],
       [0.31409688, 1.02251859],
       [0.9118464 , 0.30015916],
       ...,
       [0.59010972, 1.1923449 ],
       [0.78634504, 0.72386495],
       [0.44065885, 0.81296181]])